In [1]:
import numpy as np
from packaging import version
import subprocess
import sys
# Following code required owing to the issue here https://github.com/h2oai/h2o-3/issues/16462 , once numpy 2 is supported the code can be removed
# Check if current NumPy is >= 2
if version.parse(np.__version__) >= version.parse("2.0"):
    print(f"Detected NumPy {np.__version__}, reinstalling < 2.0...")
    !pip install --force-reinstall --no-cache-dir "numpy<2"
    # Restart runtime manually
else:
    print(f"NumPy {np.__version__} is already < 2.0. No action needed.")

NumPy 1.26.4 is already < 2.0. No action needed.


In [2]:
!pip install h2o
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.0/266.0 MB 6.7 MB/s eta 0:00:00


In [3]:
import re

In [4]:
import h2o
import pandas as pd
# helper function to format grid search results

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
h2o.init(max_mem_size = "45g")


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.28" 2025-07-15; OpenJDK Runtime Environment (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1); OpenJDK 64-Bit Server VM (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.12/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp1wobur3r
  JVM stdout: /tmp/tmp1wobur3r/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp1wobur3r/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.8
H2O_cluster_version_age:,16 days
H2O_cluster_name:,H2O_from_python_unknownUser_146pym
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,45 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [7]:
cases2010to2018_h2o = h2o.import_file('/content/drive/MyDrive/Law/Model2abbaselineData_2010to2018_merged_criminal_regr.csv')
#cases2010to2018_h2o = h2o.import_file('/content/drive/MyDrive/Law/automl_2010to2018_merged_all.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [8]:
cases2010to2018_h2o = cases2010to2018_h2o[((cases2010to2018_h2o['pendency_decision_filing'] <= 100000))]

In [9]:
cases2010to2018_h2o.types

{'state_code': 'int',
 'female_defendant': 'int',
 'female_petitioner': 'enum',
 'female_adv_def': 'int',
 'female_adv_pet': 'int',
 'billable_ipc': 'enum',
 'female_judge_filing': 'enum',
 'female_judge_decision': 'enum',
 'judgediff': 'int',
 'pendency_decision_filing': 'int',
 'pendency_first_filing': 'int',
 'muslim_defendant': 'enum',
 'muslim_petitioner': 'enum',
 'muslim_adv_def': 'int',
 'muslim_adv_pet': 'int',
 'court_details': 'enum',
 'state_district': 'enum',
 'pendency_model_multiyear': 'enum',
 'filing_judge_tenure_at_filing': 'int',
 'decision_judge_tenure_at_decision': 'int',
 'act_s': 'enum',
 'section_s': 'int',
 'type_name_s': 'enum',
 'judge_position_s': 'enum',
 'number_sections_ipc_category': 'enum'}

In [10]:
#Remove all columns not available at time of case filing

# List of columns to keep
columns_to_keep = cases2010to2018_h2o.columns

columns_to_keep.remove("female_judge_decision")
columns_to_keep.remove("decision_judge_tenure_at_decision")
columns_to_keep.remove("pendency_first_filing")
columns_to_keep.remove("judgediff")
#Drop columns related to advocate and religion
columns_to_keep.remove("muslim_defendant")
columns_to_keep.remove("muslim_petitioner")
columns_to_keep.remove("muslim_adv_def")
columns_to_keep.remove("muslim_adv_pet")
columns_to_keep.remove("female_adv_def")
columns_to_keep.remove("female_adv_pet")
columns_to_keep.remove("female_defendant")
columns_to_keep.remove("female_petitioner")
columns_to_keep.remove("female_judge_filing")


# Use the assign method to drop columns
cases2010to2018_h2o = cases2010to2018_h2o[columns_to_keep]



In [11]:
cases2010to2018_h2o.types

{'state_code': 'int',
 'billable_ipc': 'enum',
 'pendency_decision_filing': 'int',
 'court_details': 'enum',
 'state_district': 'enum',
 'pendency_model_multiyear': 'enum',
 'filing_judge_tenure_at_filing': 'int',
 'act_s': 'enum',
 'section_s': 'int',
 'type_name_s': 'enum',
 'judge_position_s': 'enum',
 'number_sections_ipc_category': 'enum'}

In [12]:
cases2010to2018_h2o

state_code,billable_ipc,pendency_decision_filing,court_details,state_district,pendency_model_multiyear,filing_judge_tenure_at_filing,act_s,section_s,type_name_s,judge_position_s,number_sections_ipc_category
25,NA,3,25_2_5,25_2,ontime,2,Code of Criminal Procedure,104,NA,Judicial Magistrate Court,NA
13,NA,0,13_25_3,13_25,ontime,49,Code of Criminal Procedure,104,transfer application (criminal),Chief Judicial Magistrate,NA
17,non-bailable,0,17_31_2,17_31,ontime,26,The Indian Penal Code,130,NA,Civil Court,5 to 10
23,non-bailable,14,23_15_1,23_15,ontime,9,The Indian Penal Code,130,NA,District And Sessions Court,5 to 10
2,non-bailable,2,2_7_17,2_7,ontime,3,The Indian Penal Code,134,cc,Civil Judge Junior Division,5 to 10
3,non-bailable,5,3_18_11,3_18,ontime,12,The Indian Penal Code,134,c.c.,principal civil judge,four
4,non-bailable,31,4_2_5,4_2,delayed,12,The Indian Penal Code,134,cc,Chief Judicial Magistrate,three
3,non-bailable,1,3_25_5,3_25,ontime,17,The Indian Penal Code,134,NA,principal civil judge,three
3,non-bailable,1,3_20_1,3_20,ontime,19,The Indian Penal Code,134,a.i.r.misc,small cause court,one
3,non-bailable,4,3_20_1,3_20,ontime,4,The Indian Penal Code,134,a.i.r.misc,small cause court,one


In [13]:
cases2010to2018_h2o['section_s'] = cases2010to2018_h2o['section_s'].asfactor()
cases2010to2018_h2o['state_code'] = cases2010to2018_h2o['state_code'].asfactor()
cases2010to2018_h2o['bailable_ipc'] = cases2010to2018_h2o['billable_ipc']


In [14]:
cases2010to2018_h2o.types

{'state_code': 'enum',
 'billable_ipc': 'enum',
 'pendency_decision_filing': 'int',
 'court_details': 'enum',
 'state_district': 'enum',
 'pendency_model_multiyear': 'enum',
 'filing_judge_tenure_at_filing': 'int',
 'act_s': 'enum',
 'section_s': 'enum',
 'type_name_s': 'enum',
 'judge_position_s': 'enum',
 'number_sections_ipc_category': 'enum',
 'bailable_ipc': 'enum'}

In [15]:
cases2010to2018_h2o.head()

state_code,billable_ipc,pendency_decision_filing,court_details,state_district,pendency_model_multiyear,filing_judge_tenure_at_filing,act_s,section_s,type_name_s,judge_position_s,number_sections_ipc_category,bailable_ipc
25,NA,3,25_2_5,25_2,ontime,2,Code of Criminal Procedure,104,NA,Judicial Magistrate Court,NA,NA
13,NA,0,13_25_3,13_25,ontime,49,Code of Criminal Procedure,104,transfer application (criminal),Chief Judicial Magistrate,NA,NA
17,non-bailable,0,17_31_2,17_31,ontime,26,The Indian Penal Code,130,NA,Civil Court,5 to 10,non-bailable
23,non-bailable,14,23_15_1,23_15,ontime,9,The Indian Penal Code,130,NA,District And Sessions Court,5 to 10,non-bailable
2,non-bailable,2,2_7_17,2_7,ontime,3,The Indian Penal Code,134,cc,Civil Judge Junior Division,5 to 10,non-bailable
3,non-bailable,5,3_18_11,3_18,ontime,12,The Indian Penal Code,134,c.c.,principal civil judge,four,non-bailable
4,non-bailable,31,4_2_5,4_2,delayed,12,The Indian Penal Code,134,cc,Chief Judicial Magistrate,three,non-bailable
3,non-bailable,1,3_25_5,3_25,ontime,17,The Indian Penal Code,134,NA,principal civil judge,three,non-bailable
3,non-bailable,1,3_20_1,3_20,ontime,19,The Indian Penal Code,134,a.i.r.misc,small cause court,one,non-bailable
3,non-bailable,4,3_20_1,3_20,ontime,4,The Indian Penal Code,134,a.i.r.misc,small cause court,one,non-bailable


In [16]:
print(len(cases2010to2018_h2o))

16482391


In [17]:
cases2010to2018_train, cases2010to2018_test, cases2010to2018_valid = cases2010to2018_h2o.split_frame(ratios=[.7, .15])

### Load Model1c

In [18]:
model1c_path = "/content/drive/MyDrive/Colab Notebooks/Law/paper2&3/paperpredictivecriminal/ToShare/Results/ResultsModel1c/saved_models/drf_grid_model_1.zip"
model1c = h2o.import_mojo(model1c_path)


generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%


In [19]:
### Load Model 2a
model2a_path = "/content/drive/MyDrive/Colab Notebooks/Law/paper2&3/paperpredictivecriminal/ToShare/Results/ResultsModel2a/saved_models/DRF_1_AutoML_1_20250808_152030.zip"
model2a = h2o.import_mojo(model2a_path)

generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%


In [20]:
### Load Model 2b
model2b_path = "/content/drive/MyDrive/Colab Notebooks/Law/paper2&3/paperpredictivecriminal/ToShare/Results/ResultsModel2b/saved_models/GBM_1_AutoML_1_20250808_152037.zip"
model2b = h2o.import_mojo(model2b_path)

generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%


In [21]:
### Load Model 2 Baseline
model2_baseline = "/content/drive/MyDrive/Colab Notebooks/Law/paper2&3/paperpredictivecriminal/ToShare/Results/ResultsModel2Baseline/saved_models/DRF_1_AutoML_1_20250808_151249.zip"
model2_baseline = h2o.import_mojo(model2_baseline)

generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%


In [ ]:
import os
import random
import numpy as np
import pandas as pd
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    accuracy_score,
    f1_score,
    roc_auc_score
)
from scipy.stats import ttest_rel
import warnings
warnings.filterwarnings('ignore')

# Parameters
n_runs = 100
sample_size = 1000  # adjust if needed
threshold_months = 24  # routing threshold
prob_threshold = [0.85]  # list so we can loop

results = []

for prob in prob_threshold:
    for run in range(n_runs):

        # 1. Sample & slice H2OFrame (sorted indices to satisfy H2O row slicing)
        random_indices = sorted(random.sample(range(cases2010to2018_test.nrow), sample_size))
        test_sample = cases2010to2018_test[random_indices, :]

        # 2. Ground truth
        y_true_duration = test_sample['pendency_decision_filing'].as_data_frame().iloc[:, 0].values
        # Map 'delayed' -> 0, 'ontime' -> 1
        y_true_delay = test_sample['pendency_model_multiyear'].as_data_frame().iloc[:, 0].apply(
            lambda x: 0 if x == "delayed" else 1
        ).values

        # -------------------------------------------------
        # Helper: get Model-1c predictions + probs in batch
        # -------------------------------------------------
        model1c_pred_full = model1c.predict(test_sample).as_data_frame()
        # predicted class mapped to 0/1
        y_pred_class_1c = model1c_pred_full['predict'].apply(
            lambda x: 0 if x == 'delayed' else 1
        ).values
        # probabilities: need prob of class "ontime" (positive class = 1)
        # H2O frame typically has columns with class names; adapt if different
        y_pred_prob_1c = model1c_pred_full['ontime'].values


        # -------------------------------------------------
        # Helper: get Model-2 baseline regression predictions in batch
        # -------------------------------------------------
        y_pred_dur_baseline_full = model2_baseline.predict(test_sample).as_data_frame().iloc[:, 0].values
        # classify baseline as on-time (1) if predicted duration <= threshold
        y_pred_class_baseline = (y_pred_dur_baseline_full <= threshold_months).astype(int)
        # pseudo "probability on-time" from baseline:
        # 1.0 if <= threshold, 0.0 otherwise
        y_pred_prob_baseline = (y_pred_dur_baseline_full <= threshold_months).astype(float)

        # =================================================
        # APPROACH 0: True Delay -> Model-2 baseline
        # =================================================
        routed_preds_0 = []
        for i, true_cls in enumerate(y_true_delay):
            # both branches are model2_baseline, so no split actually
            routed_preds_0.append(y_pred_dur_baseline_full[i])
        routed_preds_0 = np.array(routed_preds_0)

        rmse_0 = np.sqrt(mean_squared_error(y_true_duration, routed_preds_0))
        mae_0 = mean_absolute_error(y_true_duration, routed_preds_0)
        acc_class_0 = accuracy_score(y_true_delay, y_true_delay)  # always 1.0
        align_0 = np.mean(y_true_delay == (routed_preds_0 <= threshold_months))

        # F1 and AUC are "perfect" here because we're routing by ground truth.
        f1_0 = f1_score(y_true_delay, y_true_delay, zero_division=0)
        try:
            auc_0 = roc_auc_score(y_true_delay, y_true_delay)
        except:
            auc_0 = np.nan

        # =================================================
        # APPROACH 1: True Delay -> Model-2a / Model-2b
        # =================================================
        routed_preds_1 = []
        for i, true_cls in enumerate(y_true_delay):
            if true_cls == 1:  # on-time
                routed_preds_1.append(model2a.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
            else:  # delayed
                routed_preds_1.append(model2b.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
        routed_preds_1 = np.array(routed_preds_1)

        rmse_1 = np.sqrt(mean_squared_error(y_true_duration, routed_preds_1))
        mae_1 = mean_absolute_error(y_true_duration, routed_preds_1)
        acc_class_1 = accuracy_score(y_true_delay, y_true_delay)  # still 1.0
        align_1 = np.mean(y_true_delay == (routed_preds_1 <= threshold_months))
        f1_1 = f1_score(y_true_delay, y_true_delay, zero_division=0)
        try:
            auc_1 = roc_auc_score(y_true_delay, y_true_delay)
        except:
            auc_1 = np.nan

        # =================================================
        # APPROACH 2: Model-2 baseline -> Model-2a / Model-2b
        # Delay predicted via baseline regression threshold
        # =================================================
        routed_preds_2 = []
        for i, pred_dur in enumerate(y_pred_dur_baseline_full):
            if pred_dur <= threshold_months:
                routed_preds_2.append(model2a.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
            else:
                routed_preds_2.append(model2b.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
        routed_preds_2 = np.array(routed_preds_2)

        rmse_2 = np.sqrt(mean_squared_error(y_true_duration, routed_preds_2))
        mae_2 = mean_absolute_error(y_true_duration, routed_preds_2)
        acc_class_2 = accuracy_score(y_true_delay, y_pred_class_baseline)
        align_2 = np.mean((y_pred_class_baseline == (routed_preds_2 <= threshold_months)))

        f1_2 = f1_score(y_true_delay, y_pred_class_baseline, zero_division=0)
        try:
            auc_2 = roc_auc_score(y_true_delay, y_pred_prob_baseline)
        except:
            auc_2 = np.nan

        # =================================================
        # APPROACH 3: Model-1c classification -> Model-2a / Model-2b
        # =================================================
        routed_preds_3 = []
        for i, pred_cls in enumerate(y_pred_class_1c):
            if pred_cls == 1:  # on-time
                routed_preds_3.append(model2a.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
            else:  # delayed
                routed_preds_3.append(model2b.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
        routed_preds_3 = np.array(routed_preds_3)

        rmse_3 = np.sqrt(mean_squared_error(y_true_duration, routed_preds_3))
        mae_3 = mean_absolute_error(y_true_duration, routed_preds_3)
        acc_class_3 = accuracy_score(y_true_delay, y_pred_class_1c)
        align_3 = np.mean(y_pred_class_1c == (routed_preds_3 <= threshold_months))

        f1_3 = f1_score(y_true_delay, y_pred_class_1c, zero_division=0)
        try:
            auc_3 = roc_auc_score(y_true_delay, y_pred_prob_1c)
        except:
            auc_3 = np.nan

        # =================================================
        # APPROACH 4: Hybrid routing (Model-1c + Model-2 baseline)
        # Choose routing + final class with probability threshold
        # =================================================
        routed_preds_4 = []
        y_pred_class_mix = []
        y_pred_prob_mix = []

        for i, (cls_1c, prob_1c, pred_dur) in enumerate(
            zip(y_pred_class_1c, y_pred_prob_1c, y_pred_dur_baseline_full)
        ):
            # decide classification/routing:
            if cls_1c == 1:  # Model-1c says on-time
                if pred_dur <= threshold_months:
                    # both agree it's on-time
                    routed_preds_4.append(model2a.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
                    y_pred_class_mix.append(1)
                    y_pred_prob_mix.append(prob_1c)
                else:
                    # disagreement -> use prob threshold
                    if prob_1c > prob:
                        routed_preds_4.append(model2a.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
                        y_pred_class_mix.append(1)
                        y_pred_prob_mix.append(prob_1c)
                    else:
                        # fall back to baseline
                        baseline_pred = y_pred_dur_baseline_full[i]
                        routed_preds_4.append(baseline_pred)
                        cls_base = 1 if baseline_pred <= threshold_months else 0
                        y_pred_class_mix.append(cls_base)
                        y_pred_prob_mix.append(1.0 if cls_base == 1 else 0.0)

            else:  # cls_1c == 0 (delayed)
                if pred_dur > threshold_months:
                    # both agree it's delayed
                    routed_preds_4.append(model2b.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
                    y_pred_class_mix.append(0)
                    y_pred_prob_mix.append(prob_1c)  # prob delayed approx
                else:
                    # disagreement -> use prob threshold
                    if (1 - prob_1c) > prob:
                        routed_preds_4.append(model2b.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
                        y_pred_class_mix.append(0)
                        y_pred_prob_mix.append(prob_1c)
                    else:
                        baseline_pred = y_pred_dur_baseline_full[i]
                        routed_preds_4.append(baseline_pred)
                        cls_base = 1 if baseline_pred <= threshold_months else 0
                        y_pred_class_mix.append(cls_base)
                        y_pred_prob_mix.append(1.0 if cls_base == 1 else 0.0)

        routed_preds_4 = np.array(routed_preds_4)
        y_pred_class_mix = np.array(y_pred_class_mix)
        y_pred_prob_mix = np.array(y_pred_prob_mix, dtype=float)

        rmse_4 = np.sqrt(mean_squared_error(y_true_duration, routed_preds_4))
        mae_4 = mean_absolute_error(y_true_duration, routed_preds_4)
        acc_class_4 = accuracy_score(y_true_delay, y_pred_class_mix)
        align_4 = np.mean(y_pred_class_mix == (routed_preds_4 <= threshold_months))

        f1_4 = f1_score(y_true_delay, y_pred_class_mix, zero_division=0)
        try:
            auc_4 = roc_auc_score(y_true_delay, y_pred_prob_mix)
        except:
            auc_4 = np.nan

        # -------------------------------------------------
        # Collect per-run metrics
        # -------------------------------------------------
        results.append({
            # Approach 0
            'RMSE_TrueDelay+Model2baseline': rmse_0,
            'MAE_TrueDelay+Model2baseline': mae_0,
            'ACC_Class_TrueDelay+Model2baseline': acc_class_0,
            'F1_TrueDelay+Model2baseline': f1_0 * 100.0,
            'AUC_TrueDelay+Model2baseline': auc_0,
            'Align_TrueDelay+Model2baseline': align_0,

            # Approach 1
            'RMSE_TrueDelay+Model2a_b': rmse_1,
            'MAE_TrueDelay+Model2a_b': mae_1,
            'ACC_Class_TrueDelay+Model2a_b': acc_class_1,
            'F1_TrueDelay+Model2a_b': f1_1 * 100.0,
            'AUC_TrueDelay+Model2a_b': auc_1,
            'Align_TrueDelay+Model2a_b': align_1,

            # Approach 2
            'RMSE_Model2Baseline+Model2a_b': rmse_2,
            'MAE_Model2Baseline+Model2a_b': mae_2,
            'ACC_Class_Model2Baseline+Model2a_b': acc_class_2,
            'F1_Model2Baseline+Model2a_b': f1_2 * 100.0,
            'AUC_Model2Baseline+Model2a_b': auc_2,
            'Align_Model2Baseline+Model2a_b': align_2,

            # Approach 3
            'RMSE_Model1c+Model2a_b': rmse_3,
            'MAE_Model1c+Model2a_b': mae_3,
            'ACC_Class_Model1c+Model2a_b': acc_class_3,
            'F1_Model1c+Model2a_b': f1_3 * 100.0,
            'AUC_Model1c+Model2a_b': auc_3,
            'Align_Model1c+Model2a_b': align_3,

            # Approach 4
            'RMSE_Model1c+Baseline+Model2a_b': rmse_4,
            'MAE_Model1c+Baseline+Model2a_b': mae_4,
            'ACC_Class_Model1c+Baseline+Model2a_b': acc_class_4,
            'F1_Model1c+Baseline+Model2a_b': f1_4 * 100.0,
            'AUC_Model1c+Baseline+Model2a_b': auc_4,
            'Align_Model1c+Baseline+Model2a_b': align_4,
        })

# After all runs
df_results = pd.DataFrame(results)

# Save raw runs
file_name = (
    "/content/drive/MyDrive/Colab Notebooks/Law/paper2&3/"
    "paperpredictivecriminal/ToShare/ablation_results_85_final.csv"
)
df_results.to_csv(file_name, index=False)

# Summary (mean and std across runs)
summary = df_results.mean().to_frame(name='Mean').join(
    df_results.std().to_frame(name='Std')
)

print(summary)


generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |████████████████████